In [1]:
import anndata
import pandas as pd
import scimap as sm
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import numpy as np
import seaborn as sns
warnings.simplefilter(action='ignore', category=FutureWarning)

Running SCIMAP  2.2.11


This script performs NEP analysis on the MI dataset with COZI using the github scimap fork https://github.com/chiarasch/scimap/

In [2]:
path_to_csv = './../../../../../../MI_heart_paper/data/cell_table_final.csv'
obs = pd.read_csv(path_to_csv)
output_path = './../../../../Comparison/20250218_results_MI/MI_COZI_normalized_knn5.csv'
NEP_method = "COZI"
label = 'COZI_knn5'

In [3]:
# exclude the specified cell types in obs
ignore_cell_types = ['exclude']
obs = obs[~obs['final_cell_type'].isin(ignore_cell_types)]

In [4]:
#create anndata object
# add marker files to it, as SpatialLDA needs them
obs['D'] = np.random.randint(1, 101, size=len(obs))
obs['E'] = np.random.randint(1, 101, size=len(obs))

# Ensure obs has a string-based index
obs.index = obs.index.astype(str)  # Convert index to string


# Load dataframe into anndata object
# the data matrix 
X = obs[['D', 'E']]
X = X.values
adata = anndata.AnnData(X)
adata.obs = obs
adata

AnnData object with n_obs × n_vars = 563180 × 2
    obs: 'fov', 'label', 'cell_size', 'X_centroid', 'Y_centroid', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'cell_meta_cluster', 'timepoint', 'region', 'region_name', 'refined_cell_type', 'final_cell_type', 'endocardial_annotation', 'exclude_annotation', 'artefact', 'distance_from_lumen', 'lumen_bin', 'size_filter', 'D', 'E'

In [5]:
# Define the spatial interaction analysis
spatial_params = {
    'adata': adata,
    'x_coordinate': 'X_centroid',
    'y_coordinate': 'Y_centroid',
    'phenotype': 'final_cell_type',
    'method': 'knn',
    'knn': 5,
    'permutation': 300,
    'imageid': 'fov',
    'subset': None,
    'pval_method': 'zscore',
    'verbose': True,
    'label': label
}

# Conditionally include normalization based on method
if NEP_method == 'SEA':
    spatial_params['normalization'] = 'total'  # Example of adding normalization for 'knn'
if NEP_method == 'COZI':
    spatial_params['normalization'] = 'conditional'  # Example of adding normalization for 'knn'
    
# Run spatial interaction analysis with the conditional parameter
spatial = sm.tl.spatial_interaction(**spatial_params)


Processing Image: ['24h_86']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['24h_83']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['Control_13']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['Control_12']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['4h_97']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['4h_96']
Identifying the 5 nearest neighbours for every cell
Mapping phenotype to ne

In [9]:
#only include columns with zscore and replace the zscore_ with empty string
spatial = adata.uns[label]
spatial = pd.DataFrame(spatial)

spatial['interaction'] = spatial['phenotype'] + '_' + spatial['neighbour_phenotype']
# remove the two columns
spatial = spatial.drop(columns=['phenotype', 'neighbour_phenotype'])
spatial = spatial.set_index('interaction')

# SEA and COZI
spatial = spatial.filter(like='zscore_', axis=1)
spatial.columns = spatial.columns.str.replace('zscore_', '')
#
# save file
spatial.T.to_csv(output_path)
spatial.T

interaction,Cardiomyocytes_Cardiomyocytes,Cardiomyocytes_Cardiomyocytes Ankrd1+,Cardiomyocytes_Endocardial cells,Cardiomyocytes_Endothelial cells,Cardiomyocytes_Fibroblasts,Cardiomyocytes_Macrophages Trem2+,Cardiomyocytes_Macrophages Trem2-,Cardiomyocytes_Mono / Macros Ccr2+,Cardiomyocytes_Neutrophils,Cardiomyocytes_Other Leukocytes,...,Smooth muscle cells_Cardiomyocytes Ankrd1+,Smooth muscle cells_Endocardial cells,Smooth muscle cells_Endothelial cells,Smooth muscle cells_Fibroblasts,Smooth muscle cells_Macrophages Trem2+,Smooth muscle cells_Macrophages Trem2-,Smooth muscle cells_Mono / Macros Ccr2+,Smooth muscle cells_Neutrophils,Smooth muscle cells_Other Leukocytes,Smooth muscle cells_Smooth muscle cells
24h_86,176.520099,162.019734,109.889322,27.833094,26.752816,11.306028,8.328768,42.216945,43.053850,5.828626,...,21.168104,45.307174,3.659930,5.360274,13.974776,-0.139876,3.799958,6.091562,-0.003011,42.257122
24h_83,128.722837,13.313470,75.423337,43.029083,18.416585,5.791608,0.541821,24.412369,17.058975,4.516578,...,10.589112,-0.838835,7.045272,8.970660,7.137713,-11.805232,4.047045,2.987958,-0.194486,43.467896
Control_13,1.798334,8.235773,105.698105,27.348766,19.311019,10.015951,8.300272,12.470867,-0.057735,6.317959,...,0.088437,-50.218783,-5.966145,7.379736,3.494551,-52.892078,-0.149951,-0.765078,0.081787,29.208349
Control_12,35.780927,29.925643,120.249504,22.493817,26.265731,15.482124,0.000000,6.572986,5.801265,-0.220637,...,-0.669156,-25.756719,4.846169,4.180764,2.011542,3.464583,-0.055543,-1.312065,1.302700,10.987014
4h_97,103.596131,113.957647,111.367223,27.228600,18.941252,22.271857,4.667859,24.465711,55.035078,7.806050,...,13.326605,-55.429773,8.626338,8.888200,17.404243,-0.111473,8.556223,11.202141,-0.160438,41.135113
4h_96,153.717688,65.096857,77.318742,30.699287,22.211399,9.790799,-0.390541,17.123810,40.703605,21.052413,...,3.889015,-0.439315,7.724760,6.226458,0.138693,-3.735416,0.985976,12.369959,-0.224498,29.820677
48h_79,79.200038,34.970519,63.372366,56.512713,20.875393,19.756823,0.925870,11.091689,14.186736,8.033280,...,7.275661,-57.328546,-1.792329,7.130504,10.322918,0.002587,2.866245,3.977176,0.527941,36.952312
48h_76,124.767918,22.882418,38.437525,29.533997,13.378934,22.142196,10.724257,6.504887,7.217684,9.746190,...,6.626549,-0.726140,7.559597,7.533406,6.471357,-0.204536,3.441817,0.399560,4.129704,25.146576
Control_14,71.612709,7.016014,141.582451,22.975120,28.056843,20.375288,4.865221,29.246817,0.000000,-0.115888,...,0.139404,-0.552170,6.684972,7.887816,3.792111,-0.382957,0.050987,-1.081665,0.700639,52.787482


In [9]:
### NOT COMITTED YET, DECIDE IF ADDING OR NOT
# get how many cells are in each sample
total_cells = adata.obs['fov'].value_counts()
total_cells

fov
Control_14    79235
48h_79        68854
4h_97         67357
Control_12    66998
24h_86        66735
Control_13    66357
24h_83        57839
4h_96         51840
48h_76        37965
Name: count, dtype: int64

In [11]:
#only include columns with zscore and replace the zscore_ with empty string
spatial = adata.uns[label]
spatial = pd.DataFrame(spatial)

spatial['interaction'] = spatial['phenotype'] + '_' + spatial['neighbour_phenotype']
# remove the two columns
spatial = spatial.drop(columns=['phenotype', 'neighbour_phenotype'])
spatial = spatial.set_index('interaction')

# SEA and COZI
spatial = spatial.filter(like='zscore_', axis=1)
spatial.columns = spatial.columns.str.replace('zscore_', '')


# normalize scores by the squareroot of the total number of cells in that samples
# Normalize the df by the squareroot of the total number of cells stored in total_cells. Normalize by the correct samples
for col in spatial.columns:
    if col in total_cells:
        spatial[col] = spatial[col] / np.sqrt(total_cells[col])
# save file
spatial.T.to_csv(output_path)
spatial.T

interaction,Cardiomyocytes_Cardiomyocytes,Cardiomyocytes_Cardiomyocytes Ankrd1+,Cardiomyocytes_Endocardial cells,Cardiomyocytes_Endothelial cells,Cardiomyocytes_Fibroblasts,Cardiomyocytes_Macrophages Trem2+,Cardiomyocytes_Macrophages Trem2-,Cardiomyocytes_Mono / Macros Ccr2+,Cardiomyocytes_Neutrophils,Cardiomyocytes_Other Leukocytes,...,Smooth muscle cells_Cardiomyocytes Ankrd1+,Smooth muscle cells_Endocardial cells,Smooth muscle cells_Endothelial cells,Smooth muscle cells_Fibroblasts,Smooth muscle cells_Macrophages Trem2+,Smooth muscle cells_Macrophages Trem2-,Smooth muscle cells_Mono / Macros Ccr2+,Smooth muscle cells_Neutrophils,Smooth muscle cells_Other Leukocytes,Smooth muscle cells_Smooth muscle cells
24h_86,0.683309,0.627178,0.425382,0.107742,0.103560,0.043766,0.032241,0.163422,0.166661,0.022563,...,0.081942,0.175384,0.014168,0.020750,0.054096,-0.000541,0.014710,0.023580,-0.000012,0.163577
24h_83,0.535236,0.055358,0.313614,0.178917,0.076577,0.024082,0.002253,0.101508,0.070932,0.018780,...,0.044030,-0.003488,0.029295,0.037300,0.029679,-0.049087,0.016828,0.012424,-0.000809,0.180742
Control_13,0.006981,0.031971,0.410321,0.106168,0.074966,0.038882,0.032222,0.048412,-0.000224,0.024526,...,0.000343,-0.194950,-0.023161,0.028648,0.013566,-0.205328,-0.000582,-0.002970,0.000317,0.113387
Control_12,0.138236,0.115615,0.464571,0.086902,0.101475,0.059814,0.000000,0.025394,0.022413,-0.000852,...,-0.002585,-0.099508,0.018723,0.016152,0.007771,0.013385,-0.000215,-0.005069,0.005033,0.042447
4h_97,0.399165,0.439089,0.429107,0.104914,0.072982,0.085815,0.017986,0.094268,0.212055,0.030077,...,0.051349,-0.213576,0.033238,0.034247,0.067060,-0.000430,0.032968,0.043163,-0.000618,0.158497
4h_96,0.675136,0.285909,0.339588,0.134833,0.097554,0.043002,-0.001715,0.075209,0.178772,0.092463,...,0.017081,-0.001929,0.033928,0.027347,0.000609,-0.016406,0.004330,0.054330,-0.000986,0.130974
48h_79,0.301829,0.133272,0.241510,0.215368,0.079555,0.075293,0.003528,0.042270,0.054065,0.030615,...,0.027727,-0.218477,-0.006831,0.027174,0.039340,0.000010,0.010923,0.015157,0.002012,0.140824
48h_76,0.640341,0.117438,0.197271,0.151576,0.068664,0.113639,0.055040,0.033385,0.037043,0.050020,...,0.034009,-0.003727,0.038798,0.038663,0.033213,-0.001050,0.017664,0.002051,0.021195,0.129059
Control_14,0.254408,0.024925,0.502980,0.081621,0.099674,0.072384,0.017284,0.103901,0.000000,-0.000412,...,0.000495,-0.001962,0.023749,0.028022,0.013472,-0.001360,0.000181,-0.003843,0.002489,0.187531
